In [1]:
# Importarea tuturor bibliotecilor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from torch import optim

In [2]:
# Importul de biblioteci legate de rețelele neuronale
import torch
import torch.nn as nn
import os
from os import listdir
from os.path import isfile, join
from torch.utils import data
import torchvision
import torchvision.transforms as transforms
import PIL.Image as Image
from math import e
import torch.nn.functional as F
import itertools
import random
from torchvision import datasets, transforms

In [3]:
# Definim setul de date Dataset
class SNN_Dataset(data.Dataset):
    def __init__(self, root, ME_DIR, NOT_ME_DIR):
        self.root = root
        ME_PATHS = listdir(ME_DIR)
        NOT_ME_PATHS = listdir(NOT_ME_DIR)
        ME_PATHS = [join(ME_DIR, img_path) for img_path in ME_PATHS]
        NOT_ME_PATHS = [join(NOT_ME_DIR, img_path) for img_path in NOT_ME_PATHS]
        ME_PAIRS = list(itertools.permutations(ME_PATHS, 2))
        self.TRIPLETS = list(itertools.product(ME_PAIRS, NOT_ME_PATHS))
        self.TRIPLETS = [list(triplet) for triplet in self.TRIPLETS]
        for i in range(len(self.TRIPLETS)):
            self.TRIPLETS[i] = list(self.TRIPLETS[i])
            self.TRIPLETS[i][0] = list(self.TRIPLETS[i][0])
            self.TRIPLETS[i][0].append(self.TRIPLETS[i][1])
            self.TRIPLETS[i].pop(-1)
            self.TRIPLETS[i] = self.TRIPLETS[i][0]
        random.shuffle(self.TRIPLETS)
    def __getitem__(self, index):
        img_triplet = [Image.open(img).convert('RGB') for img in self.TRIPLETS[index]]
        img_triplet = [transforms.Scale((244, 244))(img) for img in img_triplet]
        img_triplet = [transforms.ToTensor()(img) for img in img_triplet]
        return img_triplet[0], img_triplet[1], img_triplet[2]
    def __len__(self):
        return len(self.TRIPLETS)

In [4]:
snn_dataset = SNN_Dataset(r'D:\Olesea\2021\IA Curs integrat\IA Practica\IA\Proiectul meu\SNN-main',r'D:\Olesea\2021\IA Curs integrat\IA Practica\IA\Proiectul meu\SNN-main\photos\ME', r'D:\Olesea\2021\IA Curs integrat\IA Practica\IA\Proiectul meu\SNN-main\photos\NOT_ME')

In [5]:
# Crearea tabelului de date.
snn_loader = torch.utils.data.DataLoader(
    snn_dataset,
    batch_size=1,
    num_workers=2
)

In [6]:
# Creare CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.maxpool1 = nn.MaxPool2d(3, stride=1)
        self.conv1_dropuot = nn.Dropout2d(0.5)
        self.comv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv2_dropuot = nn.Dropout2d(0.5)
        self.maxpool2 = nn.MaxPool2d(3, stride=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3_dropout = nn.Dropout2d(0.5)
        self.maxpool3 = nn.MaxPool2d(3, stride=1)
        self.linear1 = nn.Linear(3625216, 32)
        self.linear2 = nn.Linear(32, 16)
    def forward(self, x):
        out = self.conv1_dropuot(self.maxpool1(self.conv1(x)))
        out = self.conv2_dropuot(self.maxpool2(self.comv2(out)))
        out = self.conv3_dropout(self.maxpool3(self.conv3(out)))
        out = out.view(out.size(0), -1)
        out = self.linear1(out)
        out = self.linear2(out)
        return out

In [7]:
# Crearea modelului
model = CNN()

def loos(tested_out, known_out, non_obj_out, alpha):
    ''' This function is calcultating the loss
        :param tested_out: tensor
            The linear representation of the tested image
        :param known_out: tensor
            The linear representation of the knwon image
        :param non_obj_out: tensor
            The linear representation of the random image
        :param alpha: float
            The senzivity parameter
    '''
    norm1 = torch.norm(tested_out - known_out, p=2)
    norm2 = torch.norm(tested_out - non_obj_out, p=2)
    return max(norm1 - norm2 + alpha, torch.zeros(1, requires_grad=True))
# Definirea optimizatorului
optimizer = optim.SGD(model.parameters(), lr=0.000001, momentum=0.5)

In [8]:
print(len(snn_dataset))

270


In [9]:
# Training process - rularea procesului
for epoch in range(10):
    print("Epoch {}".format(epoch))
    train_loss = 0.0
    for triplet in snn_dataset:
        out1 = model(triplet[0].unsqueeze(1).permute(1, 0, 2, 3))
        out2 = model(triplet[1].unsqueeze(1).permute(1, 0, 2, 3))
        out3 = model(triplet[2].unsqueeze(1).permute(1, 0, 2, 3))
        loss = loos(out1, out2, out3, alpha=0.5)
        loss.backward()
        optimizer.step()
        print(loss)

Epoch 0


c:\users\olesea\appdata\local\programs\python\python37\lib\site-packages\torchvision\transforms\transforms.py:310: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
c:\users\olesea\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor(0.5719, grad_fn=<AddBackward0>)
tensor(0.9767, grad_fn=<AddBackward0>)
tensor(0.7175, grad_fn=<AddBackward0>)
tensor(0.7712, grad_fn=<AddBackward0>)
tensor(0.0571, grad_fn=<AddBackward0>)
tensor(1.1879, grad_fn=<AddBackward0>)
tensor(0.3180, grad_fn=<AddBackward0>)
tensor(0.2088, grad_fn=<AddBackward0>)
tensor(0.3613, grad_fn=<AddBackward0>)
tensor(1.5217, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor(6.5094, grad_fn=<AddBackward0>)
tensor(2.5320, grad_fn=<AddBackward0>)
tensor(1.6516, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor(3.7777, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor(3.4016, grad_fn=<AddBackward0>)
tensor(4.8976, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor(5.9792, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor(

tensor(35.9385, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor(51.7122, grad_fn=<AddBackward0>)
tensor(627.8506, grad_fn=<AddBackward0>)
tensor(62.7456, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor(146.9673, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor(28.5217, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor(210.0109, grad_fn=<AddBackward0>)
tensor(195.4245, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor(103.5284, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor(111.0963, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor(77.9197, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor(423.4373, grad_fn=<AddBackward0>)
tensor(172.4080, grad_fn=<AddBackward0>)
tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)
tensor([0.

tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBack

...

tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBack

tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBack

tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBack

tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBack

tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBack

In [10]:
# Salvarea modelului
torch.save(model, 'Olesea.pt')